In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('./net.csv')
df

,time,status
0,1706802120,on
1,1706802185,on
2,1706802251,on
3,1706802301,on
4,1706802385,off
...,...,...
3208,1706994600,on
3209,1706994660,on
3210,1706994720,on
3211,1706994780,on


In [7]:
df['time'] = pd.to_datetime(df['time'], unit='s')
df['hours'] = df['time'].dt.hour
df['minutes'] = df['time'].dt.minute

# Encode the 'status' column
df['status'] = df['status'].map({'on': 1, 'off': 0})

In [8]:
df

,time,status,hours,minutes
0,2024-02-01 15:42:00,1,15,42
1,2024-02-01 15:43:05,1,15,43
2,2024-02-01 15:44:11,1,15,44
3,2024-02-01 15:45:01,1,15,45
4,2024-02-01 15:46:25,0,15,46
...,...,...,...,...
3208,2024-02-03 21:10:00,1,21,10
3209,2024-02-03 21:11:00,1,21,11
3210,2024-02-03 21:12:00,1,21,12
3211,2024-02-03 21:13:00,1,21,13


In [9]:
X = df[['hours', 'minutes']]
y = df['status']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [14]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Assuming you have trained a linear regression model named 'model'
y_pred_linear = model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)

print(f'Mean Squared Error for Linear Regression: {mse_linear}')

# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_pred_binary = (y_pred_linear > 0.5).astype(int)

# Calculate accuracy
accuracy_linear = accuracy_score(y_test, y_pred_binary)

print(f'Accuracy for Linear Regression: {accuracy_linear:.2%}')

Mean Squared Error for Linear Regression: 0.17771519924221457
Accuracy for Linear Regression: 73.72%


In [ ]:
# Example prediction
new_data = pd.DataFrame({'hours': [19], 'minutes': [30]})
prediction = model.predict(new_data)
print(f'Predicted Status: {prediction}')

In [ ]:
new_time = pd.date_range(start=df['time'].min(), end=df['time'].max(), freq='15min')
new_data = pd.DataFrame({'time': new_time, 'hours': new_time.hour, 'minutes': new_time.minute})

# Predict probabilities for the new data
new_data['probability'] = model.predict(new_data[['hours', 'minutes']])

# Plot the chart
plt.figure(figsize=(12, 6))
plt.plot(new_data['time'], new_data['probability'], label='Predicted Probability')
plt.scatter(df[df['status'] == 1]['time'], df[df['status'] == 1]['status'], color='green', label='Actual - Internet ON', marker='o')
plt.scatter(df[df['status'] == 0]['time'], df[df['status'] == 0]['status'], color='red', label='Actual - Internet OFF', marker='o')

plt.title('Probability of Internet Being Down Over Time')
plt.xlabel('Time')
plt.ylabel('Probability')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Filter data for February 2nd
specific_day = new_data[new_data['time'].dt.date == pd.Timestamp('2024-02-02').date()]

# Plot the chart
plt.figure(figsize=(12, 6))
plt.plot(specific_day['time'], specific_day['probability'], marker='o', linestyle='-', color='blue')
plt.scatter(df[(df['time'].dt.date == pd.Timestamp('2024-02-02').date()) & (df['status'] == 1)]['time'],
            df[(df['time'].dt.date == pd.Timestamp('2024-02-02').date()) & (df['status'] == 1)]['status'],
            color='green', label='Actual - Internet ON', marker='o')
plt.scatter(df[(df['time'].dt.date == pd.Timestamp('2024-02-02').date()) & (df['status'] == 0)]['time'],
            df[(df['time'].dt.date == pd.Timestamp('2024-02-02').date()) & (df['status'] == 0)]['status'],
            color='red', label='Actual - Internet OFF', marker='o')

plt.title('Probability of Internet Being Down on February 2nd')
plt.xlabel('Time')
plt.ylabel('Probability')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Use the original data with status encoded as 0 (off) and 1 (on)
X = df[['hours', 'minutes']]
y = df['status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the logistic regression model
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)